# DSP实验（三）使用CORDIC算法求角的sin值和cos值
## 1.加载overlay

In [20]:
from pynq import Overlay

In [21]:
overlay = Overlay("./cordic.bit")

In [22]:
cordic = overlay.cordic_wrap_0

In [23]:
pi = 3.1415926535

## 2.输入角度（角度制）

In [24]:
theta = 45

## 3.查看IP核地址

In [25]:
overlay.ip_dict

{'cordic_wrap_0/s_axi_CTRL': {'fullpath': 'cordic_wrap_0/s_axi_CTRL',
  'type': 'xilinx.com:hls:cordic_wrap:1.0',
  'bdtype': None,
  'state': None,
  'addr_range': 65536,
  'phys_addr': 1073741824,
  'mem_id': 's_axi_CTRL',
  'memtype': 'REGISTER',
  'gpio': {},
  'interrupts': {},
  'parameters': {'C_S_AXI_CTRL_ADDR_WIDTH': '5',
   'C_S_AXI_CTRL_DATA_WIDTH': '32',
   'C_S_AXI_CONTROL_ADDR_WIDTH': '6',
   'C_S_AXI_CONTROL_DATA_WIDTH': '32',
   'C_M_AXI_GMEM_ID_WIDTH': '1',
   'C_M_AXI_GMEM_ADDR_WIDTH': '64',
   'C_M_AXI_GMEM_DATA_WIDTH': '32',
   'C_M_AXI_GMEM_AWUSER_WIDTH': '1',
   'C_M_AXI_GMEM_ARUSER_WIDTH': '1',
   'C_M_AXI_GMEM_WUSER_WIDTH': '1',
   'C_M_AXI_GMEM_RUSER_WIDTH': '1',
   'C_M_AXI_GMEM_BUSER_WIDTH': '1',
   'C_M_AXI_GMEM_USER_VALUE': '0x00000000',
   'C_M_AXI_GMEM_PROT_VALUE': '"000"',
   'C_M_AXI_GMEM_CACHE_VALUE': '"0011"',
   'C_M_AXI_GMEM_ENABLE_ID_PORTS': 'false',
   'C_M_AXI_GMEM_ENABLE_USER_PORTS': 'false',
   'Component_Name': 'design_1_cordic_wrap_0_12',
   

## 4.分配内存供IP使用

In [26]:
from pynq import allocate
s_buffer = allocate(shape=(1,), dtype='i4')
c_buffer = allocate(shape=(1,), dtype='i4')

## 5.配置IP
输入输出变量均定义为ap_fixed<12,2>类型，因此在写入IP核之前对角度乘以2的10次方，并对输出结果除以2的10次方。

In [27]:
theta_buffer = int(theta/180*pi*1024)
cordic.s_axi_CTRL.write(16,theta_buffer)
cordic.s_axi_control.write(0x10,s_buffer.physical_address)
cordic.s_axi_control.write(0x1c,c_buffer.physical_address)

## 6.启动IP

In [28]:
import time

cordic.s_axi_CTRL.write(0x00, 0x01)
start_time = time.time()
while True:
    reg = cordic.s_axi_CTRL.read(0x00)
    if reg != 1:
        break
end_time = time.time()

print("耗时：{}s".format(end_time - start_time))

耗时：0.0008554458618164062s


## 7.输出CORDIC结果

正弦值：

In [29]:
s_buffer[0]/1024

0.7060546875

余弦值：

In [30]:
c_buffer[0]/1024

0.70703125

## 8.计算相对误差


In [31]:
import math

正弦值误差：

In [32]:
abs((s_buffer[0]/1024-math.sin(theta/180*pi))/math.sin(theta/180*pi))

0.001487885137987689

余弦值误差:

In [33]:
abs((c_buffer[0]/1024-math.cos(theta/180*pi))/math.cos(theta/180*pi))

0.00010681725084355169